In [7]:
import pandas as pd

data_path = f"data/rate_of_sale_may_2023.snappy.parquet"

data_dict_path = f"data/rate_of_sale_data_dict.csv"

data = pd.read_parquet(data_path)

data_dict_df = pd.read_csv(data_dict_path)

In [2]:
data.head(100)

,stock_item_id,last_date_seen,first_date_seen,days_to_sell,first_retailer_asking_price,last_retailer_asking_price,can_home_deliver,reviews_per_100_advertised_stock_last_12_months,segment,seats,...,odometer_reading_miles,first_registration_date,attention_grabber,manufacturer_approved,price_indicator_rating,adjusted_retail_amount_gbp,predicted_mileage,number_of_images,advert_quality,postcode_area
0,52ae009b671ab58b3d4ff109a9fbdcf8d847de0fa190e1...,2023-05-05,2021-03-25,771,6995,6495.0,False,3.9,Independent,5.0,...,65000,2004-05-07,*IMMACULATE**FULL HISTORY*,False,NOANALYSIS,NaN,NaN,50,NaN,AL
1,32b1bac6934b1f64ff43cffa9df5aa296ead8143c36f9f...,2023-05-09,2021-05-25,714,13725,14995.0,False,NaN,Franchise,5.0,...,16018,2019-11-30,Sports Styling | Great Economy,True,GOOD,14848.0,26078.0,15,57.0,HP
2,21703d22d87eaa95c4dc81a60ba2c8cbe3b90ab659292c...,2023-05-12,2021-11-26,532,15499,13999.0,False,0.2,Independent,5.0,...,31093,2018-03-08,"Sat Nav,Leather,Auto,Euro 6",False,GREAT,14571.0,34732.0,22,61.0,SR
3,661acafc271373946cea7d30ac7f34257404ab89a1ad33...,2023-05-16,2022-02-17,453,10995,9995.0,False,7.9,Franchise,5.0,...,79000,2015-07-02,Viewing by APPOINTMENT ONLY,False,FAIR,9349.0,65684.0,30,61.0,FY
4,638216dc92410d965b416fea5b3cec9ca903368795fdde...,2023-05-04,2022-03-21,409,46000,37500.0,False,6.8,Franchise,5.0,...,10214,2022-03-03,Reserve Online,True,GOOD,37055.0,11765.0,22,48.0,LE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2f8d08de1a927b1e41e480971f757ba11070f8e68b2fd7...,2023-05-09,2023-02-01,97,13169,12475.0,False,6.2,Franchise,5.0,...,16745,2019-12-19,NAV/F&R SENSORS/12M WARRANTY!!,False,LOW,13551.0,23846.0,57,59.0,KT
96,1bf6b523def182e9479b20394a2e7b73fc289c4c634263...,2023-05-02,2023-02-02,89,24238,25349.0,True,17.5,Independent,5.0,...,15397,2020-01-04,APPLE CAR PLAY BLUETOOTH DAB,False,GOOD,24872.0,28230.0,25,44.0,DE
97,5590368a6797d2d873992f387c95196426f20dccc3f878...,2023-05-13,2023-02-02,100,10240,9921.0,False,0.1,Franchise,4.0,...,82731,2014-10-22,PAN-ROOF I AUTO I,False,GREAT,10270.0,72691.0,98,41.0,WS
98,f44869e0c1fec38fcde2d463fb4272591ce59604f1de9e...,2023-05-02,2023-02-02,89,6995,5995.0,False,1.3,Independent,5.0,...,50423,2014-03-12,Cruise- Black Leather-FSH,False,LOW,6589.0,59916.0,22,62.0,CF


In [9]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor

# Selecting features and target variable
features = ['first_retailer_asking_price', 'last_retailer_asking_price', 'can_home_deliver',
            'odometer_reading_miles', 'manufacturer_approved', 'reviews_per_100_advertised_stock_last_12_months', 'segment', 'seats', 'price_indicator_rating', 'adjusted_retail_amount_gbp', 'predicted_mileage', 'number_of_images', 'advert_quality']
target = 'days_to_sell'

# Dropping rows with missing target values
data_cleaned = data.dropna(subset=[target])

# Separating features and target
X = data_cleaned[features]
y = data_cleaned[target]

# Handling categorical and numerical features separately
categorical_features = ['can_home_deliver', 'manufacturer_approved', 'segment', 'price_indicator_rating']
numerical_features = ['first_retailer_asking_price', 'last_retailer_asking_price', 'odometer_reading_miles', 'reviews_per_100_advertised_stock_last_12_months', 'seats', 'adjusted_retail_amount_gbp', 'predicted_mileage', 'number_of_images', 'advert_quality']

# Creating transformers for numerical and categorical data
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])


# Combining transformers into a ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)])

# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Creating a pipeline with preprocessing and model
model = Pipeline(steps=[('preprocessor', preprocessor),
                        ('regressor', RandomForestRegressor(n_estimators=100, random_state=42))])


# Training the model
model.fit(X_train, y_train)

# We'll evaluate the model's performance next
model

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['first_retailer_asking_price',
                                                   'last_retailer_asking_price',
                                                   'odometer_reading_miles',
                                                   'reviews_per_100_advertised_stock_last_12_months',
                                                   'seats',
                                                   'adjusted_retail_amount_gbp',
                                                   'predicted_mileage',
                                                   'number_of_images',
                                                   'advert_quality']),
                                                 ('cat',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['can_home_deliver',
                                                   'manufacturer_approved',
                                                   'segment',
                                                   'price_indicator_rating'])])),
                ('regressor', RandomForestRegressor(random_state=42))])

In [10]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


# Making predictions on the test set
y_pred = model.predict(X_test)

# Calculating the Mean Absolute Error
mae = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error: {mae}")

# Calculating the Mean Squared Error
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

# Calculating the R-squared value
r2 = r2_score(y_test, y_pred)
print(f"R-squared: {r2}")

Mean Absolute Error: 28.210804387195417
Mean Squared Error: 2328.9604601300507
R-squared: 0.25274341333545225


In [31]:
# Example data for one row (ensure the order and structure match your model's training data)
#data = {'first_retailer_asking_price': [5795],  # Example values
        # 'last_retailer_asking_price': [5795],
        # 'can_home_deliver': ['no'],
        # 'odometer_reading_miles': [46585],
        # 'manufacturer_approved': ['no']}

# Convert the data into a DataFrame
#single_row_df = pd.DataFrame(data)

# Use the model to make a prediction for the single row
row_to_test = 8

prediction = model.predict(data_cleaned.iloc[[row_to_test]][features])

real = data_cleaned.iloc[row_to_test]['days_to_sell']

print(f"Predicted days to sell: {prediction[0]}")
print(f"Actual days to sell: {real}")

Predicted days to sell: 242.87
Actual days to sell: 327


In [12]:
X_test

,first_retailer_asking_price,last_retailer_asking_price,can_home_deliver,odometer_reading_miles,manufacturer_approved
203413,14000,13700.0,False,49308,False
10746,5795,5795.0,False,46585,False
32285,10999,11299.0,False,19570,False
184165,25310,21990.0,False,8907,True
197383,31450,31450.0,False,20000,False
...,...,...,...,...,...
84625,3795,3795.0,False,108000,False
94928,16300,16000.0,False,33176,False
124625,15989,15989.0,False,19741,False
99314,30950,31950.0,False,17049,True


In [17]:
data_cleaned.iloc[10746]

stock_item_id                                      1e942ca6c4c86c95d7f7e94c004833b138bcec3ef6f1be...
last_date_seen                                                                            2023-05-03
first_date_seen                                                                           2023-04-21
days_to_sell                                                                                      12
first_retailer_asking_price                                                                     5795
last_retailer_asking_price                                                                    5795.0
can_home_deliver                                                                               False
reviews_per_100_advertised_stock_last_12_months                                                  1.5
segment                                                                                  Independent
seats                                                                                      